<a href="https://colab.research.google.com/github/sejongsmarcle/2023_Spring_Kaggle_Study/blob/main/week3/1%ED%8C%80/%EA%B8%B8%ED%83%9C%ED%98%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 데이터 및 라이브러리 불러오기

In [2]:
!apt -qq -y install fonts-nanum > /dev/null

import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

fontpath='/usur/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font=fm.FontProperties(fname=fontpath,size=9)

#fm._rebuild()
#그래프에 retina display 적용
%config inlineBackend.figure_format='retina'

#colab의 한글 폰트 설정
plt.rc('font',family='NanumBarunGothic')


In [3]:
import pandas as pd
import lightgbm as lgb
import folium

In [4]:
#데이터 불러오기
train=pd.read_csv('/content/drive/MyDrive/버스 캐글/train.csv')
test=pd.read_csv('/content/drive/MyDrive/버스 캐글/test.csv')
submission=pd.read_csv('/content/drive/MyDrive/버스 캐글/submission_제출양식.csv')

# 2. 탐색적 자료분석

In [5]:
#train 상위 5개 확인
train.head()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
0,0,2019-10-15,405136001,7997025,360-1,33.456267,126.551750,제주대학교입구,06시,266.0,제대마을,33.457724,126.554014,24
1,1,2019-10-15,405136001,7997025,360-1,33.457724,126.554014,제대마을,06시,333.0,제대아파트,33.458783,126.557353,36
2,2,2019-10-15,405136001,7997025,360-1,33.458783,126.557353,제대아파트,06시,415.0,제주대학교,33.459893,126.561624,40
3,3,2019-10-15,405136001,7997025,360-1,33.479705,126.543811,남국원(아라방면),06시,578.0,제주여자중고등학교(아라방면),33.484860,126.542928,42
4,4,2019-10-15,405136001,7997025,360-1,33.485662,126.494923,도호동,07시,374.0,은남동,33.485822,126.490897,64


In [6]:
#test 상위 5개 확인
test.head()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude
0,210457,2019-10-29,405136001,7997025,360-1,33.457724,126.554014,제대마을,07시,333.0,제대아파트,33.458783,126.557353
1,210458,2019-10-29,405136001,7997025,360-1,33.458783,126.557353,제대아파트,07시,415.0,제주대학교,33.459893,126.561624
2,210459,2019-10-29,405136001,7997025,360-1,33.478867,126.483833,한라중학교/부영아파트,08시,417.0,대림2차아파트,33.478850,126.488350
3,210460,2019-10-29,405136001,7997025,360-1,33.478850,126.488350,대림2차아파트,08시,334.0,연동대림1차아파트,33.480700,126.489933
4,210461,2019-10-29,405136001,7997025,360-1,33.480700,126.489933,연동대림1차아파트,08시,550.0,케이티앤지,33.482077,126.485355


In [7]:
#submission 상위 5개 확인
submission.head()

,id,next_arrive_time
0,210457,0
1,210458,0
2,210459,0
3,210460,0
4,210461,0


In [8]:
#train 끝 5개 확인
train.tail()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
210452,210452,2019-10-28,405328102,7983486,281-2,33.255783,126.577450,비석거리,21시,528.0,삼아아파트,33.251896,126.574417,96
210453,210453,2019-10-28,405328102,7983486,281-2,33.248595,126.568527,동문로터리,21시,280.0,매일올레시장 7번입구,33.249753,126.565959,50
210454,210454,2019-10-28,405328102,7983486,281-2,33.251891,126.560303,서귀포시 구 버스터미널,21시,114.0,아랑조을거리 입구,33.251084,126.559551,16
210455,210455,2019-10-28,405328102,7983486,281-2,33.251084,126.559551,아랑조을거리 입구,21시,223.0,평생학습관,33.249504,126.558068,38
210456,210456,2019-10-28,405328102,7983486,281-2,33.248487,126.511195,대륜동주민센터,21시,189.0,서귀포버스터미널(가상정류소),33.249091,126.509224,24


In [9]:
#train과 test의 날짜의 최대최소 확인(날짜가 train과 test가 이어짐)
print(train['date'].min())
print(train['date'].max())
print(test['date'].min())
print(test['date'].max())

2019-10-15
2019-10-28
2019-10-29
2019-11-05


In [10]:
#데이터의 형태 확인
print(train.shape)
print(test.shape)
print(submission.shape)

(210457, 14)
(91774, 13)
(91774, 2)


In [11]:
#train의 결측값 여부 확인(결측값 없음)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210457 entries, 0 to 210456
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                210457 non-null  int64  
 1   date              210457 non-null  object 
 2   route_id          210457 non-null  int64  
 3   vh_id             210457 non-null  int64  
 4   route_nm          210457 non-null  object 
 5   now_latitude      210457 non-null  float64
 6   now_longitude     210457 non-null  float64
 7   now_station       210457 non-null  object 
 8   now_arrive_time   210457 non-null  object 
 9   distance          210457 non-null  float64
 10  next_station      210457 non-null  object 
 11  next_latitude     210457 non-null  float64
 12  next_longitude    210457 non-null  float64
 13  next_arrive_time  210457 non-null  int64  
dtypes: float64(5), int64(4), object(5)
memory usage: 22.5+ MB


In [12]:
#test의 결측값 여부 확인(결측값 없음)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91774 entries, 0 to 91773
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               91774 non-null  int64  
 1   date             91774 non-null  object 
 2   route_id         91774 non-null  int64  
 3   vh_id            91774 non-null  int64  
 4   route_nm         91774 non-null  object 
 5   now_latitude     91774 non-null  float64
 6   now_longitude    91774 non-null  float64
 7   now_station      91774 non-null  object 
 8   now_arrive_time  91774 non-null  object 
 9   distance         91774 non-null  float64
 10  next_station     91774 non-null  object 
 11  next_latitude    91774 non-null  float64
 12  next_longitude   91774 non-null  float64
dtypes: float64(5), int64(3), object(5)
memory usage: 9.1+ MB


In [13]:
#train의 산술적 정보 요약 확인
train.describe()

,id,route_id,vh_id,now_latitude,now_longitude,distance,next_latitude,next_longitude,next_arrive_time
count,210457.000000,2.104570e+05,2.104570e+05,210457.000000,210457.000000,210457.000000,210457.000000,210457.000000,210457.000000
mean,105228.000000,4.052491e+08,7.988694e+06,33.434528,126.603451,490.256100,33.434711,126.603687,85.380824
std,60753.847139,9.132404e+04,6.774077e+03,0.102350,0.123961,520.563932,0.102224,0.123838,85.051170
min,0.000000,4.051360e+08,7.983000e+06,33.244382,126.473300,97.000000,33.244382,126.473300,6.000000
25%,52614.000000,4.051365e+08,7.983093e+06,33.325283,126.523900,291.000000,33.325283,126.524550,44.000000
50%,105228.000000,4.053201e+08,7.983431e+06,33.484667,126.551050,384.000000,33.484860,126.551050,66.000000
75%,157842.000000,4.053201e+08,7.997041e+06,33.500197,126.650322,542.000000,33.500228,126.650322,102.000000
max,210456.000000,4.053281e+08,7.997124e+06,33.556167,126.935188,7461.000000,33.556167,126.935188,2996.000000


In [14]:
#train의 now_station의 숫자 종류 확인
len(train['now_station'].unique())

348

In [15]:
#route_id와 vh_id의 중복값을 없애고 노선별 버스 수 확인
train[['route_id','vh_id']].drop_duplicates().groupby('route_id').count()

,vh_id
route_id,
405136001,10
405136002,5
405136007,1
405136012,3
405136521,6
405136522,15
405320111,3
405320112,5
405320113,1


In [16]:
#route_id와 vh_id의 중복값을 없애고 노선별 도착시간 확인
train[['route_id','next_arrive_time']].drop_duplicates().groupby('route_id').mean()

,next_arrive_time
route_id,
405136001,214.493827
405136002,494.006873
405136007,151.703125
405136012,521.729231
405136521,596.943262
405136522,439.278846
405320111,206.722543
405320112,228.140704
405320113,163.738462


folium.Map(): location 옵션에 있는 좌표를 중심으로 지도 시각화

In [17]:
#지도 시각화
coord=[35.1559361,129.1384361]
map_osm=folium.Map(location=coord)
map_osm

In [18]:
#zoom_start: 확대 범위
map_osm=folium.Map(location=coord,zoom_start=12)
map_osm

In [19]:
#tiles: 지도 스타일
map_osm=folium.Map(location=coord,tiles='Stamen Terrain')
map_osm

In [20]:
#tiles: 지도 스타일
map_osm=folium.Map(location=coord,tiles='Stamen Toner')
map_osm

In [21]:
coord=[35.2335123,129.0810047]

map_osm=folium.Map(location=coord,zoom_start=12)
#마커 생성 후 지도에 추가
folium.Marker(coord,popup='부산대학교',tooltip='부산대학교 tooltip').add_to(map_osm)

map_osm.save('index.html')

map_osm

In [22]:
map_osm=folium.Map(location=coord,zoom_start=12)
#마커 생성 후 지도에 추가, 아이콘 추가
folium.Marker(coord,popup='부산대학교',icon=folium.Icon(icon='home')).add_to(map_osm)


map_osm

In [23]:
#icon:아이콘 형태 변경 가능
map_osm=folium.Map(location=[37.541,126.986],zoom_start=11)
folium.Marker([37.566345,126.977893],popup='서울특별시청',icon=folium.Icon(color='red',icon='info-sign')).add_to(map_osm)
folium.Circle([37.4600,126.9519],tooltip='서울대학교',radius=1000,color="green").add_to(map_osm)
folium.CircleMarker([37.5662,126.9386],radius=10,popup='연세대학교',color="black",fill_color="crimson").add_to(map_osm)
map_osm


In [24]:
#number_of_sides: n각형,radius:반지름, fill_opacity: 투명도 
map_osm=folium.Map(location=[37.541,126.986],zoom_start=11)
folium.RegularPolygonMarker([37.566345,126.977893],popup='서울특별시청',fill_color='red',color='red',number_of_sides=3,radius=30,fill_opacity=0.2).add_to(map_osm)
folium.RegularPolygonMarker([37.4600,126.9519],tooltip='서울대학교',fill_color="green",color="green",number_of_sides=4,radius=10,fill_opacity=0.2).add_to(map_osm)
folium.RegularPolygonMarker([37.5662,126.9386],popup='연세대학교',fill_color="blue",color='blue',number_of_sides=6,radius=10).add_to(map_osm)
map_osm


In [25]:
map_data=train[['route_id','now_latitude','now_longitude','now_station']]
map_bus_route=map_data[map_data['route_id']==405136521].drop_duplicates("now_station")

map_osm=folium.Map(location=[33.4134,126.5190],zoom_start=10.5)
#각 노선의 버스 정류장을 마커로 찍기
for item in map_bus_route.index:
    lat = map_bus_route.loc[item,'now_latitude']
    long = map_bus_route.loc[item,'now_longitude']
    
    folium.Marker([lat,long], popup = map_bus_route.loc[item,'now_station'],
                      icon = folium.Icon(color = 'red', icon = 'info-sign')
                      ).add_to(map_osm)
map_osm.save('index.html')

map_osm

# 3. 데이터 전처리

In [26]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210457 entries, 0 to 210456
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                210457 non-null  int64  
 1   date              210457 non-null  object 
 2   route_id          210457 non-null  int64  
 3   vh_id             210457 non-null  int64  
 4   route_nm          210457 non-null  object 
 5   now_latitude      210457 non-null  float64
 6   now_longitude     210457 non-null  float64
 7   now_station       210457 non-null  object 
 8   now_arrive_time   210457 non-null  object 
 9   distance          210457 non-null  float64
 10  next_station      210457 non-null  object 
 11  next_latitude     210457 non-null  float64
 12  next_longitude    210457 non-null  float64
 13  next_arrive_time  210457 non-null  int64  
dtypes: float64(5), int64(4), object(5)
memory usage: 22.5+ MB


In [27]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91774 entries, 0 to 91773
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               91774 non-null  int64  
 1   date             91774 non-null  object 
 2   route_id         91774 non-null  int64  
 3   vh_id            91774 non-null  int64  
 4   route_nm         91774 non-null  object 
 5   now_latitude     91774 non-null  float64
 6   now_longitude    91774 non-null  float64
 7   now_station      91774 non-null  object 
 8   now_arrive_time  91774 non-null  object 
 9   distance         91774 non-null  float64
 10  next_station     91774 non-null  object 
 11  next_latitude    91774 non-null  float64
 12  next_longitude   91774 non-null  float64
dtypes: float64(5), int64(3), object(5)
memory usage: 9.1+ MB


In [28]:
#train셋의 now_arrive_time을 숫자로 변경
train['now_arrive_time']=train['now_arrive_time'].str.slice(stop=-1).astype('int')

In [30]:
#test셋의 now_arrive_time을 숫자로 변경
test['now_arrive_time']=test['now_arrive_time'].str.slice(stop=-1).astype('int')

In [31]:
#train셋의 now_arrive_time을 int형으로 변경되었는지 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210457 entries, 0 to 210456
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                210457 non-null  int64  
 1   date              210457 non-null  object 
 2   route_id          210457 non-null  int64  
 3   vh_id             210457 non-null  int64  
 4   route_nm          210457 non-null  object 
 5   now_latitude      210457 non-null  float64
 6   now_longitude     210457 non-null  float64
 7   now_station       210457 non-null  object 
 8   now_arrive_time   210457 non-null  int64  
 9   distance          210457 non-null  float64
 10  next_station      210457 non-null  object 
 11  next_latitude     210457 non-null  float64
 12  next_longitude    210457 non-null  float64
 13  next_arrive_time  210457 non-null  int64  
dtypes: float64(5), int64(5), object(4)
memory usage: 22.5+ MB


In [32]:
#test셋의 now_arrive_time을 int형으로 변경되었는지 확인
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91774 entries, 0 to 91773
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               91774 non-null  int64  
 1   date             91774 non-null  object 
 2   route_id         91774 non-null  int64  
 3   vh_id            91774 non-null  int64  
 4   route_nm         91774 non-null  object 
 5   now_latitude     91774 non-null  float64
 6   now_longitude    91774 non-null  float64
 7   now_station      91774 non-null  object 
 8   now_arrive_time  91774 non-null  int64  
 9   distance         91774 non-null  float64
 10  next_station     91774 non-null  object 
 11  next_latitude    91774 non-null  float64
 12  next_longitude   91774 non-null  float64
dtypes: float64(5), int64(4), object(4)
memory usage: 9.1+ MB


# 4. 변수 선택 및 모델 구축

In [33]:
 train.head()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
0,0,2019-10-15,405136001,7997025,360-1,33.456267,126.551750,제주대학교입구,6,266.0,제대마을,33.457724,126.554014,24
1,1,2019-10-15,405136001,7997025,360-1,33.457724,126.554014,제대마을,6,333.0,제대아파트,33.458783,126.557353,36
2,2,2019-10-15,405136001,7997025,360-1,33.458783,126.557353,제대아파트,6,415.0,제주대학교,33.459893,126.561624,40
3,3,2019-10-15,405136001,7997025,360-1,33.479705,126.543811,남국원(아라방면),6,578.0,제주여자중고등학교(아라방면),33.484860,126.542928,42
4,4,2019-10-15,405136001,7997025,360-1,33.485662,126.494923,도호동,7,374.0,은남동,33.485822,126.490897,64


In [34]:
#학습시킬 때 사용할 변수를 features에 넣고, 예측할 변수를 target에 넣기
features=['now_latitude','now_longitude','now_arrive_time','distance']
target=['next_arrive_time']

In [36]:
#X_train,X_test,y_train에 맞는 데이터 넣기
X_train,X_test,y_train=train[features],test[features],train[target]

# 5. 모델 학습 및 검증

In [38]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [47]:
#여러 모델을 딕셔너리 형태로 만들기
model_dict={
    'linear':LinearRegression(),
    'rf':RandomForestRegressor(random_state=0,n_jobs=-1),
    'lgbm':lgb.LGBMRegressor(random_state=0)
}

In [48]:
#딕셔너리의 key 확인
model_dict.keys()

dict_keys(['linear', 'rf', 'lgbm'])

In [51]:
#모델 훈련 및 예측, 예측 결과를 model_result에 각각 저장
model_result={}
for key in model_dict.keys():
  print("###훈련 중 ###")
  model_dict[key].fit(X_train,y_train)
  print("###예측 중 ###")
  model_result[key]=model_dict[key].predict(X_test)

###훈련 중 ###
###예측 중 ###
###훈련 중 ###


<ipython-input-51-4b539f4ac83f>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_dict[key].fit(X_train,y_train)


###예측 중 ###
###훈련 중 ###
###예측 중 ###


In [52]:
#훈련 결과 확인
model_result

{'linear': array([[ 80.75162733],
        [ 86.59637449],
        [ 96.03730695],
        ...,
        [ 93.13399549],
        [ 72.93764136],
        [103.28646517]]),
 'rf': array([ 35.36650507,  45.12237491, 135.88134374, ..., 126.37972036,
         31.84467172,  88.46184286]),
 'lgbm': array([ 42.12400643,  60.24878305, 120.81241189, ..., 105.81387439,
         30.82952485, 111.66787953])}

# 6. 제출

In [53]:
#각 모델별 제출 파일 복사
lr_submit=submission.copy()
rf_submit=submission.copy()
lgbm_submit=submission.copy()

In [54]:
#각 모델별 제출 파일에 예측 결과 저장
lr_submit['next_arrive_time']=model_result['linear']
rf_submit['next_arrive_time']=model_result['rf']
lgbm_submit['next_arrive_time']=model_result['lgbm']

In [55]:
#제출파일 확인
lr_submit.head()

,id,next_arrive_time
0,210457,80.751627
1,210458,86.596374
2,210459,96.037307
3,210460,89.402964
4,210461,105.457036


In [57]:
#제출파일 만들어 저장
lr_submit.to_csv('lr_submit.csv',index=False)
rf_submit.to_csv('rf_submit.csv',index=False)
lgbm_submit.to_csv('lgbm_submit.csv',index=False)